# Constructing a Dobble card deck

The card deck of a Dobble game is defined by the rules

1. Any two cards have exactly one symbol in common.
2. Any two symbols appear together on exactly one card.

It turns out that these requirements have an exact geometrical analogue. In two-dimensional affine geometry, the following two rules hold:

1. Any two non-parallel lines meet in exactly one point.
2. Any two points lie on (define) exactly one line. 

Since these characteristics depend only on the underlying vector-space, they particularly hold for vector-spaces over *finite fields* $\mathbb{F}_q$ with $q$ elements. So the strategy to construct a Dobble card game is as follows. Associate "card" with "line" and "symbol" with "point" and consider the affine plane $\mathbb{A}^2_q$ over the vector space $\mathbb{F}_q^2$. Next, construct every line in $\mathbb{A}^2_q$. Consider therefore that every line is defined by an equation
$
\begin{equation}
    \mathbf{w}\cdot\mathbf{x} + b = 0 
\end{equation}
$
for some $\mathbf{w} \in \mathbb{F}_q^2$ and $b\in \mathbb{F}_q$. In other words, each line is defined by a tuple $(\mathbf{w}, b)$ where $\mathbf{w}$ ranges over all different "directions" of the vector space. It is easily verified that 
$$
    W := \left\{ (\bar 0,\bar 1)^T \right\} \cup \left\{ (\bar 1, \bar 0)^T, (\bar 1, \bar 2)^T, \ldots, (\bar 1,\overline{q-1})^T \right\} \subset \mathbb{F}_q^2
$$

is a complete and linear independent set representing all the directions in $\mathbb{A}_q ^2$. 
With basic linear algebra one can show that the $(q+1) \cdot q$ tuples $(\mathbf{w}, b) \in W \times \mathbb{F}_q$ really define all the lines in 
$\mathbb{A}_q^2$. Note that for every vector $\mathbf{x} = (x_1,x_2)^T$ in equation (1) it is true that if you fix any component $x_1$ or $x_2$, there is exactly one solution for the other one. It follows that every line contains exactly $q$ points. 

Now, two different lines intersect in one point if they are _not parallel_, i.e. they have different norm vectors $\mathbf{w}$, 
whereas two lines with the same directions don't have a point in common in the affine plane. 
To make sure that also in the parallel case two lines share an equal point, we assign a new element 
$p_{\mathbf{w}} \not\in\mathbb{A}_q^2$, called a "point at infinity", to every line with direction $\mathbf{w}$. With this convention, any two lines have exactly one point in common, whether they share the same direction or not. Finally, we add another line containing all the "points at infinity". This guarantees that any two points lie on exactly one line.  (Technically, we are moving on to the _projective plane_ over $\mathbb{F}_q$, but for our purpose we can treat these infinity points and lines as simple _ad hoc_ postulations because at this point of the argument we don't need any algebraic structure of the point set anymore).

This finishes the construction of the Dobble card game. Substituting "card" with "line" and "point" with symbol, we see that all the required properties for the Dobble desk are satisfied. Numberwise, the desk has the following properties:

* Every card contains $q+1$ symbols. (Every line contains $q$ normal points plus one "point at infinity".)
* There are $q^2 + q + 1$ different symbols. (The $q^2$ points in $\mathbb{A}_q^2$ plus $q+1$ "points at infinity".)
* There are $q^2 + q + 1$ different cards. (The $q^2 + q$ lines in $\mathbb{A}_q^2$ plus one "line at infinity".)

Note that the construction depends on the existence of a field $\mathbb{F}_q$ with $q$ elements. Such a field exists if and only if $q$ is a prime power, i.e. $q=p^n$ for some prime number $p$ and natural exponent $n$. For $n=1$ this is just the residual ring $\mathbb{Z}/p\mathbb{Z}$ where the arithmetic is defined by calculating everything modulo $p$. But in general, for $n>1$, the field $\mathbb{F}_{p^n}$ is _not_ isomorphic to 
$\mathbb{Z}/p^n\mathbb{Z}$. The arithmetic in these general fields is somehow less obvious, and that's the reason we need a specialiced library for that. 

In [213]:
import random
import galois
import numpy as np

class Dobble:

    # q is the order of the finite field. Therefore, q has to be a prime power, q=p^n for some prime p.
    # Every card will contain q+1 symbols
    def __init__(self, q):
        self.q = q
        self.order = q**2+q+1
        self.GF = galois.GF(q)
        # In a finite affine vector space over a field of order q there are:
        #   -- q^2 points
        #   -- q+1 different directions: w=(0,1),(1,0),...,(1,q-1)
        self.points = [self.GF((x,y)) for x in range(q) for y in range(q)]
        dirs = [(0,1)] + [(1,m) for m in range(q)]
        
        # Each element of the lines-list represents the class of parallel lines for the direction w,
        # an element has the structure 
        # 
        # [w, [[points for b=0],[points for b=1],...,[points for b=q-1]]]
        # 
        # The q point lists for every w are initialized with the "point at infinity", which by convention are denoted with (-w_0,-w_1)
        self.lines = [[self.GF(w), [[-np.array(w)] for _ in range(q)]] for w in dirs]

        # Assign the points
        for x in self.points:
            for line in self.lines:
                # Note that x and line[0] are instances of a galois fieldarray,
                # i.e. that the dot-product is calculated over the field F_q
                b = -np.dot(x, line[0])
                line[1][b].append(x)
        
        # Add line at infinity
        self.lines.append([np.array([0,0]), [[-np.array(w) for w in dirs]]])

    def play(self):
        i,j = random.sample(range(self.order), 2)

        line_a = self.lines[i // len(self.lines)][1][i % self.q]
        line_b = self.lines[j // len(self.lines)][1][j % self.q]
        
        common_points = []

        for x in line_a:
            for y in line_b:
                if x[0]==y[0] and x[1]==y[1]:
                    common_points.append(x)
        
        print("New Game")
        print(f"Card 1: {[x.tolist() for x in line_a]}")
        print(f"Card 2: {[x.tolist() for x in line_b]}")
        print(F"Common symbol: {[x.tolist() for x in common_points]}")
        print("")

In [243]:
dobble = Dobble(4)

for _ in range(5):
    dobble.play()

for line in dobble.lines:
    for p in line[1]:
        print([x.tolist() for x in p])
    

New Game
Card 1: [[-1, -2], [0, 0], [1, 3], [2, 1], [3, 2]]
Card 2: [[-1, -2], [0, 1], [1, 2], [2, 0], [3, 3]]
Common symbol: [[-1, -2]]

New Game
Card 1: [[-1, 0], [3, 0], [3, 1], [3, 2], [3, 3]]
Card 2: [[-1, -1], [0, 2], [1, 3], [2, 0], [3, 1]]
Common symbol: [[3, 1]]

New Game
Card 1: [[0, -1], [0, 2], [1, 2], [2, 2], [3, 2]]
Card 2: [[-1, -2], [0, 0], [1, 3], [2, 1], [3, 2]]
Common symbol: [[3, 2]]

New Game
Card 1: [[-1, 0], [3, 0], [3, 1], [3, 2], [3, 3]]
Card 2: [[-1, 0], [0, 0], [0, 1], [0, 2], [0, 3]]
Common symbol: [[-1, 0]]

New Game
Card 1: [[-1, 0], [3, 0], [3, 1], [3, 2], [3, 3]]
Card 2: [[0, -1], [0, 1], [1, 1], [2, 1], [3, 1]]
Common symbol: [[3, 1]]

[[0, -1], [0, 0], [1, 0], [2, 0], [3, 0]]
[[0, -1], [0, 1], [1, 1], [2, 1], [3, 1]]
[[0, -1], [0, 2], [1, 2], [2, 2], [3, 2]]
[[0, -1], [0, 3], [1, 3], [2, 3], [3, 3]]
[[-1, 0], [0, 0], [0, 1], [0, 2], [0, 3]]
[[-1, 0], [1, 0], [1, 1], [1, 2], [1, 3]]
[[-1, 0], [2, 0], [2, 1], [2, 2], [2, 3]]
[[-1, 0], [3, 0], [3, 1], [3,